### Car sales program:
##### Carry out the following tasks in JupyterLab using the prepared DataFrame from the previous chapter:
##### Conduct an elbow-test and determine the optimal number of clusters.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Seaborn is another alternative plotting library. 
import seaborn as sns
sns.set()

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
car_dropmodel_price = pd.read_csv("car_dropmodel_price.csv")
car_dropmodel_price

Now, K-Means is an unsupervised machine learning method. This means there is no need for a train-test split. KMeans also allows you to use the ENTIRE dataset.

However, 1 thing we need to do is to normalize the data set, but only the numerical variables. We know that Price is the only numerical variable we want to normalize since normalizing year does not make sense

In [ ]:
# Drop all the Price cells that have NA in them
car_clean_price = car_dropmodel_price.dropna(axis = 0, how = "any")
car_clean_price

In [ ]:
len(car_clean_price["Make"].str.get_dummies(sep=', ').sum())

In [ ]:
len(car_clean_price["Category"].str.get_dummies(sep=', ').sum())

In [ ]:
print(car_clean_price["Make"].unique().tolist())
print("\n")
print(car_clean_price["Category"].unique().tolist())

#### What about the non-numerical data? Will that work in K-Means? The answer is NO!

##### We have to convert all non-numerical data to binary values using dummy variables.

Think about it - how does K-Means (which is based on euclidean distance) know how far the word "Audi" is from "Mercedes"?

In [ ]:
# I originally tried this but the outcome was pretty messy. Have a go in any case

#X_km = pd.get_dummies(X_km)
#X_km_norm = pd.get_dummies(X_km_norm)

In [ ]:
columns_rename1 = { 
    "Audi":1, "Chevrolet":2, "Cadillac":3, "Acura":4, "BMW":5, "Chrysler":6, "Ford":7, "Buick":8, "INFINITI":9,
    "GMC":10, "Honda":11, "Hyundai":12, 'Jeep' : 13, 'Genesis': 14, 'Dodge': 16, 'Jaguar': 17, 'Kia': 18, 'Land Rover': 19,
    'Lexus': 20, 'Mercedes-Benz': 21, 'Mitsubishi': 22, 'Lincoln': 23, 'MAZDA': 24, 'Nissan': 25, 'MINI': 26, 'Porsche': 27,
    'Ram': 28, 'Subaru': 29, 'Toyota': 30, 'Volkswagen': 31, 'Volvo': 32, 'Alfa Romeo':33
}

car_clean_price = car_clean_price.replace({'Make':columns_rename1})
#print(car_clean_price["Make"].unique().tolist())

columns_rename2 = {'SUV':1, 'Sedan':2, 'Coupe':3, 'Pickup':4, 'Van':5, 'Convertible':6, 'Hatchback':7, 'Wagon':8}
car_clean_price = car_clean_price.replace({'Category':columns_rename2})
#print(car_clean_price["Category"].unique().tolist())

car_clean_price

In [ ]:
car_clean_price.to_csv("car_clean_price.csv", index = False)

In [ ]:
# Here all we are doing is selecting the column we want to normalize; the other steps are just to note the column and row labels
numvar_list = ['Price']
DF_model_toNorm = car_clean_price[numvar_list]
DF_model_toNorm_colnam = list(DF_model_toNorm.columns.values)
DF_model_toNorm_rownam = list(DF_model_toNorm.index)
DF_model_toNorm

In [ ]:
# This normalizes the data set
scaler = preprocessing.StandardScaler()
scaler.fit(DF_model_toNorm)
DF_model_Norm = scaler.transform(DF_model_toNorm)

# Now we convert to a pandas array
car_norm_price = pd.DataFrame(DF_model_Norm, columns = DF_model_toNorm_colnam, index = DF_model_toNorm_rownam)
car_norm_price

In [ ]:
# Now we recreate the original dataframe
car_norm_price = car_norm_price.add_suffix("_norm")
numnormvar_list = car_norm_price.columns.values
df_car_final = pd.concat([car_clean_price, car_norm_price], axis = 1)
df_car_final

In [ ]:
# First lets drop the normalized values for the elbow plot; the name is stored in "numnormvar_list"
X_km = df_car_final.drop(numnormvar_list, axis = 1)

# This second data set is for once we have optimized the number of clusters
X_km_norm = df_car_final.drop(numvar_list, axis = 1)

X_km_norm.columns = X_km_norm.columns.str.replace("_norm", "")
X_km_norm

##### Construct a K-Means clustering solution with the number of clusters obtained from the previous task.


### Lets start to do the elbow plot

In [ ]:
distortions = []

min_el = 1
max_el = 10

# A loop runs thorugh your min and max clusters. 
# We dont complicate the K-Means at this stage
# We store an "intertia" number which is the measurement of the distortions in different K clustering solutions

for i in range (min_el,max_el):
    km = KMeans(n_clusters= i, init = "k-means++")
    km.fit(X_km)
    distortions.append(km.inertia_)    

In [ ]:
# elbow curve
fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (5,3), dpi = 150)
plt.plot(range (min_el,max_el), distortions, marker='o', markersize=5,
    markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
plt.grid(True)
plt.xlabel('Number of clusters')
plt.ylabel('Average within-cluster sum of squares')
plt.title('Elbow for KMeans clustering')

#### Looks like 2 is the optimal number of clusters. Lets do the K means proper now

In [ ]:
# Fitting the K-Means with the optimal number of cluster
k = 2
km = KMeans(n_clusters=k, init = "k-means++", random_state = 0)
km_fit = km.fit(X_km_norm)

# Predicting the Classificaiton of the Data
y_pred = km.fit_predict(X_km_norm)
display(y_pred)

##### Explore the estimated clustering solution by cross-tabulating the cluster index with the clustering criteria.

In [ ]:
# The predicted results should now be converted to a pandas dataframe

y_pred_data = pd.DataFrame({"cluster":y_pred},index = X_km_norm.index)
y_pred_data

In [ ]:
car_pred = pd.concat([X_km , y_pred_data ], axis = 1)
car_pred

In [ ]:
#car_pred[['Price', 'cluster']].groupby(by = ['cluster']).any().transpose()
pd.crosstab(car_pred['Price'], car_pred['cluster'], normalize='index', margins = True)

##### Plot the clustering solution as a 2-dimensional scatter plot.

In [ ]:
filtered_label0 = car_pred['cluster' == 0]
filtered_label1 = car_pred['cluster' == 1]

fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (5,3), dpi = 150)
plt.plot(filtered_label0[:, 0], filtered_label0[:, 1], Price, marker='o', markersize=5,
    markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
plt.plot(filtered_label1[:, 0], filtered_label1[:, 1], Price, marker='o', markersize=5,
    markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
plt.grid(True)
plt.xlabel('Number of clusters')
plt.ylabel('Average within-cluster sum of squares')
plt.title('Elbow for KMeans clustering')